In [6]:
import re
import requests
from tqdm import tqdm
from libs.index_loader import loader
from libs.req import req
from libs.extract import extract
from libs.cleaner import cleaner
from libs.writer import Writer

In [7]:
requester = req()
my_loader = loader()
my_cleaner = cleaner()
my_writer = Writer()

__配置文件__

In [8]:
level = 2 # 爬取树的层数
clean = False # 循环处理是否清洗
input_urls = my_loader.load_file('./urls_files/index_urls.txt')
pattern = re.compile('<a.*?href="http://.*?".*?>.*?</a>')
raw_name = './urls_files/raw_urls.txt'
processed_name = './urls_files/processed_urls.txt'


33it [00:00, 34336.90it/s]

index_urls load complete


__爬虫主程序__

In [9]:
def crawler(input_urls, level):
    if level == 0:
        print('End Crawling')
        return
    error_urls = []
    str_combine = ''
    if clean:
        input_urls = my_cleaner.clean_url(input_urls, input_urls)
    for url in tqdm(input_urls, desc='Crawling URLs'):
        try:
            str_combine = str_combine + requester.req_url(url)
        except Exception:
            error_urls.append(url)
            continue

    items = re.findall(pattern, str_combine)
    next_url = []
    for item in tqdm(items, desc='Extracting URLs'):
        extractor = extract()
        dict_ex = extractor.url_extract_less(item)
        for url_text in dict_ex:
            next_url.append(url_text)
    with open(raw_name, 'w') as file:
        for url in next_url:
            file.write(url + '\n')
    print(f'{len(error_urls)} URL went wrong:{error_urls}',)
    print(f'Complete')
    crawler(next_url, level - 1)

__开始爬取__

In [10]:
crawler(input_urls, level)

Crawling URLs: 100%|██████████| 33/33 [03:07<00:00,  5.68s/it]


Request timed out


Extracting URLs: 100%|██████████| 361/361 [00:00<00:00, 812.48it/s] 


11 URL went wrong:['http://magnet.chongbuluo.com', 'https://www.9178.work', 'https://oo1.win', 'https://web.geekji.cn', 'https://anyi.life', 'http://daohangwan.com', 'http://lxqnsys.com', 'https://www.24kdh.com', 'http://qiachu.com', 'http://321ku.com', 'http://byr.wiki']
Complete


Crawling URLs: 100%|█████████▉| 1878/1883 [1:01:36<01:41, 20.28s/it]

Request timed out


Extracting URLs: 100%|██████████| 32981/32981 [00:29<00:00, 1115.19it/s]


147 URL went wrong:['https://www.zhihu.com/', 'http://www.cpta.com.cn/', 'https://ipcrs.pbccrc.org.cn/', 'https://www.qimao.com/', 'http://www.mafengwo.cn/', 'http://www.lianjia.com/', 'http://www.baixing.com/', 'http://www.cwl.gov.cn/fcpz/yxjs/fc3d/', 'http://www.pbc.gov.cn/', 'https://outlook.live.com/', 'https://beian.miit.gov.cn/', 'https://www.mafengwo.cn/', 'https://outlook.live.com/', 'https://www.lanzou.com/', 'https://bing.ioliu.cn/', 'https://jnav.com/xuexi', 'https://www.vmovier.com/channel', 'https://www.hdhhh.com/', 'https://yigekuang.cn/', 'https://beian.miit.gov.cn/', 'http://www.qcenglish.com/', 'http://www.wordlm.com/', 'https://icoalert.com/', 'https://www.zuankuo.com', 'http://www.rigengjihua.cn/', 'https://lingkou.com', 'https://tsgvpn.hynu.edu.cn:8888/', 'https://magi.com/', 'https://www.xiaoduyu.com', 'https://www.52pojie.cn/misc.php?mod=ranklist&type=thread&view=replies&orderby=thisweek', 'https://weekly.75.team/', 'https://dribbble.com/', 'https://www.5118.com/'

__清洗__

In [11]:
my_cleaner.clean_url(raw_name, processed_name)

__处理__

In [12]:
new_lines = []

with open(processed_name, 'r') as file:
    lines = file.readlines()
    for line in lines:
        new_line = line.strip() + '/robots.txt\n'
        new_lines.append(new_line)

with open(processed_name, 'w') as file:
    file.writelines(new_lines)

__写入__

In [14]:
timeout_list = []
no_response_list = []

with open(processed_name, 'r') as file:
    lines = file.readlines()
    for line in tqdm(lines,desc='Crawling URLs'):
        try:
            response = requests.get(line, timeout=10)
            if response.status_code == 200:
                if response.text.startswith('User-agent:'):
                    my_writer.parse_robots_to_csv(website_url=line, robots_text=response.text)
            else:
                no_response_list.append(line)
        except requests.exceptions.Timeout:
            timeout_list.append(line)
        except Exception as e:
            continue
    file.close()
print('complete')

Crawling URLs: 100%|██████████| 6541/6541 [1:32:36<00:00,  1.18it/s]  

compl
